In [24]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling/")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec

# Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier as RFC
# :n_estimators: The number of trees in the forest
N_ESTIMATORS = 17
# :criterion: The function to measure the quality of a split
CRITERION = "entropy"
# :min_samples_split:The minimum number of samples
# required to split an internal node
MIN_SAMPLES_SPLIT = 3

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True


# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict


In [7]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [11]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
subcategory = df['Subcategory']

## word2vec model

In [12]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
model = Word2Vec.load('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')
w2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [13]:
# Use pipes to implement steps of fit and transform method
rfc_w2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_dict)),
    ("Random Forest", RFC(n_estimators=N_ESTIMATORS, criterion=CRITERION,
                          min_samples_split=MIN_SAMPLES_SPLIT))])

## Train Random Forest Model

In [14]:
# Split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    columns, subcategory, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [20]:
# Fit Random Forest model with out train data
rfc_w2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7f9e02796c40>),
                ('Random Forest',
                 RandomForestClassifier(criterion='entropy',
                                        min_samples_split=3,
                                        n_estimators=17))])

In [21]:
# Model Evaluation
rfc_w2v.score(X_train, y_train)

0.9969418960244648

In [22]:
# Random Forest prediction based on test data
prediction = rfc_w2v.predict(X_test)
prediction

array([12, 14,  0, 12, 20,  9, 29, 10,  0,  2, 10, 28, 24, 10, 10,  3, 15,
       20, 20, 18, 14, 21,  0,  9,  9,  2,  0, 21, 28, 26,  9,  2, 12, 21,
       10, 20,  2, 16, 29, 10,  2,  7, 23, 26, 22, 17, 30,  4, 16, 20, 14,
       20, 14, 20, 28, 14,  9, 26,  9, 21, 14,  0, 10, 21, 24,  0,  9, 14,
       20, 29, 20, 10,  0,  0, 30, 10, 10, 12, 24, 20, 24,  4,  0, 10, 10,
        0,  0, 21, 20,  2, 29, 21,  0, 10,  4, 27, 10, 10,  0, 10,  2, 29,
       22, 30, 22, 10, 16, 21,  9, 20,  2, 21,  9, 10, 34,  2, 21, 20, 10,
        2, 16,  2,  9, 10, 12, 26, 14, 16, 20, 20, 17, 10, 16,  9, 10,  7,
        0, 14, 27, 15, 20, 10,  4, 20, 26, 10, 29, 26, 12, 26, 10, 12, 21,
       12, 12,  2, 10, 10, 30, 10, 20, 20,  4, 28, 28, 10, 14, 10, 12, 10,
       14, 20, 30, 28,  3, 10, 21, 10, 30,  0, 26, 14, 34, 10,  0,  0, 10,
        0, 16, 10, 10, 19, 15, 29, 24, 30, 28, 20, 30, 20, 20, 22,  0, 14,
       10, 22,  4, 22, 10, 19, 14, 12, 10, 14, 10, 23, 18, 10, 10,  0, 16,
       10, 18,  7,  7, 10

## Evaluation

In [23]:
# View evaluation metrics table
evaluation = model_evaluation_metrics(y_test, prediction)
evaluation

,Metric Labels,Metric values
0,Accuracy,0.277580
1,Precision,0.147132
2,Recall,0.149787
3,F1-Score,0.140891
4,Mean Absolute Error,7.277580
5,Mean Squared Error,110.971530


In [26]:
# Transform columns data to dict to use in MLFlow
transformation = series_pair_to_dict(evaluation, 'Metric Labels', 'Metric values')
transformation

{'Accuracy': 0.2775800711743772,
 'Precision': 0.1471319321546025,
 'Recall': 0.14978708655179243,
 'F1-Score': 0.14089063079834638,
 'Mean Absolute Error': 7.277580071174377,
 'Mean Squared Error': 110.97153024911032}

In [27]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Subcategory (Random Forest, Word2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'word2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "n_estimators": N_ESTIMATORS,
                  "criterion": CRITERION,
                  "min_samples_split": MIN_SAMPLES_SPLIT
                 }
    log_params(parameters)
    log_metrics(transformation)